<a href="https://colab.research.google.com/github/msvt37/HW_11_Risky_Business/blob/main/credit_risk_resampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Credit Risk Resampling Techniques

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV into DataFrame

In [4]:
# Load the data
file_path = Path('/content/drive/MyDrive/Colab Notebooks/ClassificationHW/Starter_Code/Resources/lending_data.csv')
df = pd.read_csv(file_path)
df



,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk
...,...,...,...,...,...,...,...,...,...
77531,19100.0,11.261,own,86600,0.653580,12,2,56600,high_risk
77532,17700.0,10.662,mortgage,80900,0.629172,11,2,50900,high_risk
77533,17600.0,10.595,rent,80300,0.626401,11,2,50300,high_risk
77534,16300.0,10.068,mortgage,75300,0.601594,10,2,45300,high_risk


In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(df['homeowner'])
df['homeowner'] = le.transform(df['homeowner'])

le.fit(df['loan_status'])
df['loan_status'] = le.transform(df['loan_status'])

#own = 1, rent = 2, mortg = 0
# low_risk = 1

# Split the Data into Training and Testing

In [6]:
# Create our features
X = df.drop(columns='loan_status')

# Create our target
y = df[['loan_status']]

In [7]:
# Check the balance of our target values
y['loan_status'].value_counts()

1    75036
0     2500
Name: loan_status, dtype: int64

In [8]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
X_train

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
25729,10100.0,7.402,0,50300,0.403579,4,1,20300
20176,9200.0,7.049,0,46900,0.360341,3,0,16900
55080,8800.0,6.870,0,45200,0.336283,3,0,15200
10410,8900.0,6.919,0,45700,0.343545,3,0,15700
63110,8500.0,6.731,1,43900,0.316629,3,0,13900
...,...,...,...,...,...,...,...,...
4767,12300.0,8.346,0,59100,0.492386,6,1,29100
26012,8300.0,6.634,1,43000,0.302326,2,0,13000
58637,6700.0,5.988,0,36900,0.186992,1,0,6900
23339,9200.0,7.054,0,47000,0.361702,3,0,17000


## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [9]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [10]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [11]:
# Scale the training and testing data
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Simple Logistic Regression

In [12]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [13]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9892813049736127

In [14]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  615,    10],
       [  102, 18657]])

In [15]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.86      0.98      0.99      0.92      0.99      0.98       625
          1       1.00      0.99      0.98      1.00      0.99      0.98     18759

avg / total       0.99      0.99      0.98      0.99      0.99      0.98     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [16]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_oversampled, y_overesampled = ros.fit_resample(X_train, y_train)
# View the count of target classes with Counter
Counter(y_overesampled)

Counter({0: 56277, 1: 56277})

In [17]:
# Train the Logistic Regression model using the resampled data
model2 = LogisticRegression(solver='lbfgs', random_state=1)
model2.fit(X_oversampled, y_overesampled)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [18]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred_naive = model2.predict(X_test)
balanced_accuracy_score(y_test, y_pred_naive)

0.9946414201183431

In [19]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred_naive)

array([[  622,     3],
       [  111, 18648]])

In [20]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred_naive))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.85      1.00      0.99      0.92      0.99      0.99       625
          1       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



### SMOTE Oversampling

In [21]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_smote, y_smote = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train, y_train)
# View the count of target classes with Counter
Counter(y_smote)

Counter({0: 56277, 1: 56277})

In [22]:
# Train the Logistic Regression model using the resampled data
model3 = LogisticRegression(solver='lbfgs', random_state=1)
model3.fit(X_smote, y_smote)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [23]:
# Calculated the balanced accuracy score
y_pred_smote = model3.predict(X_test)
balanced_accuracy_score(y_test, y_pred_smote)

0.9946414201183431

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_smote)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_smote))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.85      1.00      0.99      0.92      0.99      0.99       625
          1       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [25]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_clustercentr, y_clustercentr = cc.fit_resample(X_train, y_train)
# View the count of target classes with Counter
Counter(y_clustercentr)

Counter({0: 1875, 1: 1875})

In [26]:
# Train the Logistic Regression model using the resampled data
model4 = LogisticRegression(solver='lbfgs', random_state=1)
model4.fit(X_clustercentr, y_clustercentr)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [27]:
# Calculate the balanced accuracy score
y_pred_clustercentr = model4.predict(X_test)
balanced_accuracy_score(y_test, y_pred_clustercentr)

0.9932813049736127

In [28]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_clustercentr)

array([[  620,     5],
       [  102, 18657]])

In [29]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_clustercentr))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.86      0.99      0.99      0.92      0.99      0.99       625
          1       1.00      0.99      0.99      1.00      0.99      0.99     18759

avg / total       1.00      0.99      0.99      0.99      0.99      0.99     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [30]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN
sm = SMOTEENN(random_state=1)
X_smoteenn, y_smoteenn = sm.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_smoteenn)

Counter({0: 55667, 1: 55917})

In [31]:
# Train the Logistic Regression model using the resampled data
model5 = LogisticRegression(solver='lbfgs', random_state=1)
model5.fit(X_smoteenn, y_smoteenn)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [32]:
# Calculate the balanced accuracy score
y_pred_smoteenn = model5.predict(X_test)
balanced_accuracy_score(y_test, y_pred_smoteenn)

0.9946414201183431

In [33]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_smoteenn)

array([[  622,     3],
       [  111, 18648]])

In [34]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_smoteenn))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.85      1.00      0.99      0.92      0.99      0.99       625
          1       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



# Final Questions

1. Which model had the best balanced accuracy score?  It appears that several of the models had scores that were identical.  I used different variables to ensure it was not a carry-over from the earlier code.  

   Combination Over and Under:0.9946414201183431
   ClusterCentroid Undersampling: 0.9932813049736127
   Smote Oversampling: 0.9946414201183431
   Naive Random Oversampling: 0.9946414201183431

2. Which model had the best recall score?

    Recall scores appeared to be the same across the models

3. Which model had the best geometric mean score?

    As with Recall, the geometric mean appears to be the same   across the models as well.
